In [9]:
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np
import pandas as pd
import re
def mult(a,b):
    return a/b

In [10]:
mol_name = 'output_5.mol' 
mol = Chem.MolFromMolFile(mol_name)
cif_name = f'data_cif{mol_name}'
#名字是象征意义，封装时传递mol
# 以mol传递mol文件
m2 = Chem.AddHs(mol) # 加氢原子
AllChem.EmbedMolecule(m2) # 2D->3D化
AllChem.MMFFOptimizeMolecule(m2)   # 使用MMFF94最小化RDKit生成的构象
#m3 = Chem.RemoveHs(m2) # 删除氢原子
#print(Chem.MolToMolBlock(m3))
m2.SetProp('_Name', 'cyclobutane')
Chem.MolToMolFile(m2, 'output_5.txt')
CAS_file=open('output_5.txt','r',encoding='utf-8')
cas_line=CAS_file.readlines()
top_1 = pd.DataFrame()
top_2 = pd.DataFrame()
test_0 = '0  0  0  0  0  0  0  0  0  0  0  0'
my_re_eng = re.compile(r'[A-Za-z]',re.S)
my_re_num = re.compile(r'[0-9]',re.S)
a = 0
for i in range(len(cas_line)):
    res = re.findall(my_re_eng,cas_line[i])
    if test_0 in cas_line[i]:
        cif = cas_line[i].split()
        top_1.insert(i-4,column=f"atom{i-3}",value=cif)
    elif len(res) or cas_line[i]=='\n':
        pass
    else:
        a = a+1
        cif = cas_line[i].split()
        top_2.insert(0,column=f"bond{a}",value=cif)
#提取出来了必要的信息

In [11]:
#loc_mult 计算后的原子位置
rate = 40
loc = top_1[0:3]
loc = loc.astype("float")
loc_mult = loc.pipe(mult,rate) 

In [12]:
for i in range(1,loc_mult.shape[1]+1): 
    atom_num = '{0}{1}'.format(top_1['atom{}'.format(i)][3],i) #原子名，号
    atom_type = top_1['atom{}'.format(i)][3] #原子名
    atom_loc_0 = loc_mult['atom{}'.format(i)][0]
    atom_loc_0 = round(atom_loc_0, 5)     #原子位置x
    atom_loc_1 = loc_mult['atom{}'.format(i)][1]
    atom_loc_1 = round(atom_loc_1, 5)     #原子位置y
    atom_loc_2 = loc_mult['atom{}'.format(i)][2]
    atom_loc_2 = round(atom_loc_2, 5)     #原子位置z
    print(atom_num,'   ',atom_type,'   ',atom_loc_0,'   ',atom_loc_1,'   ',atom_loc_2,'   0.00000  Uiso   1.00')

F1     F     0.04574     -0.0503     0.00313    0.00000  Uiso   1.00
C2     C     0.023     -0.02573     0.00158    0.00000  Uiso   1.00
C3     C     -0.01078     -0.03353     -0.00062    0.00000  Uiso   1.00
C4     C     -0.03372     -0.0075     -0.00219    0.00000  Uiso   1.00
O5     O     -0.06693     -0.01512     -0.00434    0.00000  Uiso   1.00
C6     C     -0.023     0.02573     -0.00158    0.00000  Uiso   1.00
F7     F     -0.04574     0.0503     -0.00313    0.00000  Uiso   1.00
C8     C     0.01078     0.03353     0.00062    0.00000  Uiso   1.00
C9     C     0.03372     0.0075     0.00219    0.00000  Uiso   1.00
O10     O     0.06693     0.01512     0.00434    0.00000  Uiso   1.00
H11     H     -0.01892     -0.05938     -0.00108    0.00000  Uiso   1.00
H12     H     -0.07912     0.00601     -0.0052    0.00000  Uiso   1.00
H13     H     0.01892     0.05938     0.00108    0.00000  Uiso   1.00
H14     H     0.07912     -0.00601     0.0052    0.00000  Uiso   1.00


In [13]:
for j in range(1,top_2.shape[1]+1): #判断成键分子 、类型、及键长 
    x = 0
    ins_1 = '{0}{1}'.format(top_1['atom{}'.format(top_2['bond{}'.format(j)][0])][3],top_2['bond{}'.format(j)][0])#成键分子1
    ins_2 = '{0}{1}'.format(top_1['atom{}'.format(top_2['bond{}'.format(j)][1])][3],top_2['bond{}'.format(j)][1]) #成键分子2 
    if top_2['bond{}'.format(j)][2] == '1':#判断成键类型
        band_type = 'S'
    elif top_2['bond{}'.format(j)][2] == '2':
        band_type = 'D'
    elif top_2['bond{}'.format(j)][2] == '3':
        band_type = 'T'
    else:
        pass
    for i in range(3):  #判断成键类型
        a = loc['atom{}'.format(top_2['bond{}'.format(j)][0])][i]
        b = loc['atom{}'.format(top_2['bond{}'.format(j)][1])][i]
        c = (a-b)**2
        x = x + c
    x = x**0.5
    print(ins_1,'     ',ins_2,'      ',x,'   ','.','     ',band_type)
    
#整体第二块

F1       C2        1.3408611188337143     .       S
C2       C3        1.389734553790759     .       D
C3       C4        1.3890574394171034     .       S
C4       O5        1.3658819129046256     .       S
C4       C6        1.3967411571225357     .       D
C6       F7        1.3408611188337143     .       S
C6       C8        1.389734553790759     .       S
C8       C9        1.389061964060639     .       D
C9       O10        1.3658756056098225     .       S
C9       C2        1.3967429076247353     .       S
C3       H11        1.0840256223909104     .       S
O5       H12        0.9763505978899178     .       S
C8       H13        1.0840256223909104     .       S
O10       H14        0.9763505978899178     .       S
